In [6]:
from pathlib import Path

from pandas import DataFrame

from src import load_base_model, results_path


def process_files(
    path: Path = results_path,
):
    data = {}
    # Loops on files.
    for model_path in path.glob("*"):
        for run_path in model_path.joinpath("runs").glob("*"):
            for file_path in run_path.joinpath("load").glob("*/*/*/*/*/barystatic_rise_mean_trend.json"):
                barystatic_sea_level = load_base_model(path=file_path.parent, name=file_path.name)
                parents = str(file_path.parent).split("/")
                # Line.
                line = {
                    "model": model_path.name,
                    "run": run_path.name.split('bounded')[0],
                    "signal": parents[-5],
                    "load": parents[-4],
                    "case": parents[-3],
                    "LIA": parents[-2],
                    "continents": parents[-1],
                    "barystatic_difference_with_elastic": barystatic_sea_level["anelastic"] - barystatic_sea_level["elastic"],
                }
                for key, value in line.items():
                    if not key in data.keys(): data[key]=[]
                    data[key] += [value]
    return DataFrame(data)


# Example usage:
df = process_files()
print(df)
df.to_csv(results_path.joinpath("barystatic_sea_level_differences_with_elastic.csv"))

                                                  model  \
0     PREM_____Lambeck_2017____eta_m__ASTHENOSPHERE_...   
1     PREM_____Lambeck_2017____eta_m__ASTHENOSPHERE_...   
2     PREM_____Lambeck_2017____eta_m__ASTHENOSPHERE_...   
3     PREM_____Lambeck_2017____eta_m__ASTHENOSPHERE_...   
4     PREM_____Lambeck_2017____eta_m__ASTHENOSPHERE_...   
...                                                 ...   
5246  PREM_____Lau_2016____eta_m__ASTHENOSPHERE__3e+...   
5247  PREM_____Lau_2016____eta_m__ASTHENOSPHERE__3e+...   
5248  PREM_____Lau_2016____eta_m__ASTHENOSPHERE__3e+...   
5249  PREM_____Lau_2016____eta_m__ASTHENOSPHERE__3e+...   
5250  PREM_____Lau_2016____eta_m__ASTHENOSPHERE__3e+...   

                                                    run      signal  \
0                           __short_term_anelasticity__  ocean_load   
1                           __short_term_anelasticity__  ocean_load   
2                           __short_term_anelasticity__  ocean_load   
3      

In [12]:
from pandas import read_csv

df = read_csv("data/results/barystatic_sea_level_differences_with_elastic.csv")
df

,Unnamed: 0,model,run,signal,load,case,LIA,continents,barystatic_difference_with_elastic
0,0,PREM_____Lambeck_2017____eta_m__ASTHENOSPHERE_...,__short_term_anelasticity__,ocean_load,Frederikse,upper,without_LIA,with_opposite_load_on_continents,0.18317
1,1,PREM_____Lambeck_2017____eta_m__ASTHENOSPHERE_...,__short_term_anelasticity__,ocean_load,Frederikse,upper,without_LIA,without_opposite_load_on_continents,0.16587
2,2,PREM_____Lambeck_2017____eta_m__ASTHENOSPHERE_...,__short_term_anelasticity__,ocean_load,Frederikse,upper,with_LIA,with_opposite_load_on_continents,0.17914
3,3,PREM_____Lambeck_2017____eta_m__ASTHENOSPHERE_...,__short_term_anelasticity__,ocean_load,Frederikse,upper,with_LIA,without_opposite_load_on_continents,0.16297
4,4,PREM_____Lambeck_2017____eta_m__ASTHENOSPHERE_...,__short_term_anelasticity__,ocean_load,Frederikse,mean,without_LIA,with_opposite_load_on_continents,0.19581
...,...,...,...,...,...,...,...,...,...
5246,5246,PREM_____Lau_2016____eta_m__ASTHENOSPHERE__3e+...,long_term_anelasticity__short_term_anelasticity__,ocean_load,Frederikse,mean,with_LIA,with_opposite_load_on_continents,0.18503
5247,5247,PREM_____Lau_2016____eta_m__ASTHENOSPHERE__3e+...,long_term_anelasticity__short_term_anelasticity__,ocean_load,Frederikse,mean,with_LIA,without_opposite_load_on_continents,0.16495
5248,5248,PREM_____Lau_2016____eta_m__ASTHENOSPHERE__3e+...,long_term_anelasticity__short_term_anelasticity__,ocean_load,Frederikse,lower,without_LIA,without_opposite_load_on_continents,0.18542
5249,5249,PREM_____Lau_2016____eta_m__ASTHENOSPHERE__3e+...,long_term_anelasticity__short_term_anelasticity__,ocean_load,Frederikse,lower,with_LIA,with_opposite_load_on_continents,0.19882


In [14]:
df.groupby(["run"]).agg({'barystatic_difference_with_elastic':['mean','min', 'max', 'std']})

barystatic_difference_with_elastic  \
                                                                                mean   
run                                                                                    
__short_term_anelasticity__                                                 0.182056   
long_term_anelasticity____                                                  0.031217   
long_term_anelasticity__short_term_anelasticity__                           0.209250   

                                                                               
                                                       min      max       std  
run                                                                            
__short_term_anelasticity__                        0.13717  0.24571  0.022703  
long_term_anelasticity____                         0.00657  0.06343  0.015623  
long_term_anelasticity__short_term_anelasticity__  0.14320  0.28602  0.026709

In [ ]:
all_attributes = ["model", "run", "signal", "load", "case", "LIA", "continents"]
all_attributes.remove("LIA")
df.groupby(all_attributes).agg({'barystatic_difference_with_elastic':['mean', 'min', 'max', 'std']}).groupby(["run"]).agg({'std':['mean', 'min', 'max']})

barystatic_difference_with_elastic  \
                                                                                            mean   
run                                               LIA                                              
__short_term_anelasticity__                       with_LIA                              0.180188   
                                                  without_LIA                           0.183924   
long_term_anelasticity____                        with_LIA                              0.028153   
                                                  without_LIA                           0.034280   
long_term_anelasticity__short_term_anelasticity__ with_LIA                              0.206831   
                                                  without_LIA                           0.212149   

                                                                        \
                                                                   min   
run                                               LIA                    
__short_term_anelasticity__                       with_LIA     0.13717   
                                                  without_LIA  0.13944   
long_term_anelasticity____                        with_LIA     0.00657   
                                                  without_LIA  0.00876   
long_term_anelasticity__short_term_anelasticity__ with_LIA     0.14320   
                                                  without_LIA  0.14758   

                                                                        \
                                                                   max   
run                                               LIA                    
__short_term_anelasticity__                       with_LIA     0.23922   
                                                  without_LIA  0.24571   
long_term_anelasticity____                        with_LIA     0.05394   
                                                  without_LIA  0.06343   
long_term_anelasticity__short_term_anelasticity__ with_LIA     0.28368   
                                                  without_LIA  0.28602   

                                                                         
                                                                    std  
run                                               LIA                    
__short_term_anelasticity__                       with_LIA     0.022182  
                                                  without_LIA  0.023074  
long_term_anelasticity____                        with_LIA     0.014353  
                                                  without_LIA  0.016236  
long_term_anelasticity__short_term_anelasticity__ with_LIA     0.026444  
                                                  without_LIA  0.026753